In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ML Recomendacion_juego

#### Cargar el Conjunto de Datos

In [2]:
df1 = pd.read_csv("../data/fusionada.csv")
df1.head()

,app_name,release_date,price,item_id,developer,clasificacion,playtime_forever,user_id,items_count,recommend,sentiment_analysis,posted
0,Carmageddon Max Pack,1997,9.99,282010,Stainless Games Ltd,Action,466.0,InstigatorAU,112.0,True,1.0,2013-01-12
1,Half-Life,1998,9.99,70,Valve,Action,1395.0,EizanAratoFujimaki,391.0,True,1.0,2015-10-28
2,Half-Life,1998,9.99,70,Valve,Action,590.0,GamerFag,64.0,True,1.0,2011-01-27
3,Half-Life,1998,9.99,70,Valve,Action,5599.0,76561198020928326,110.0,True,1.0,2014-07-01
4,Half-Life,1998,9.99,70,Valve,Action,64.0,Bluegills,1280.0,True,2.0,2013-12-06


In [3]:
df1.to_parquet('data_fusionada.parquet', index=False)

#### Seleccionar Columnas Relevantes

In [5]:
columnas = ['app_name', 'item_id', 'clasificacion']
df1 = df1[columnas]
df1.head()

,app_name,item_id,clasificacion
0,Carmageddon Max Pack,282010,Action
1,Half-Life,70,Action
2,Half-Life,70,Action
3,Half-Life,70,Action
4,Half-Life,70,Action


In [7]:
# Filtrar Filas no Deseadas
df1 = df1[df1['clasificacion'] != 'Pendiente de clasificación']

#### Crear Variables Dummy

In [9]:
df_dummies = pd.get_dummies(df1, columns=['clasificacion'], prefix="")
df_dummies.head()

,app_name,item_id,_Action,_Adventure,_Casual,_Education,_Horror,_Indie,_Open World,_Puzzle,_RPG,_Racing,_Shooter,_Simulation,_Sports,_Strategy
0,Carmageddon Max Pack,282010,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Half-Life,70,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Half-Life,70,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Half-Life,70,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Half-Life,70,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [10]:
# Agrupar y Sumar
df_agrupado = df_dummies.groupby(['item_id', 'app_name'], as_index=False).sum()
df_agrupado.head()

,item_id,app_name,_Action,_Adventure,_Casual,_Education,_Horror,_Indie,_Open World,_Puzzle,_RPG,_Racing,_Shooter,_Simulation,_Sports,_Strategy
0,10,Counter-Strike,45,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,15,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,4,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,3,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Calcular la Similitud del Coseno

In [11]:
similitudes = cosine_similarity(df_agrupado.iloc[:, 2:])

#### Encontrar y Ordenar Juegos Similares

In [12]:
id_producto = 282010
idx = df_agrupado.index[df_agrupado['item_id'] == id_producto].tolist()[0]
similarity_scores = list(enumerate(similitudes[idx]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

#### Generar Recomendaciones

In [13]:
indices_similares = [i[0] for i in similarity_scores[1:6]]
juegos_recomendados = df_agrupado.iloc[indices_similares]['app_name']
juegos_recomendados

1        Team Fortress Classic
2                Day of Defeat
3           Deathmatch Classic
4    Half-Life: Opposing Force
5                     Ricochet
Name: app_name, dtype: object

# ML recomendacion_usuario

### Carga y preparar los datos

In [15]:
# Especificar las columnas relevantes para el análisis
columnas = ['user_id', 'item_id', 'recommend', 'app_name']

# Cargar el DataFrame solo con las columnas especificadas
df = pd.read_csv('../data/fusionada.csv', usecols=columnas)

# Convertir 'recommend' a valores numéricos (1 para True, 0 para False)
df['interaction'] = df['recommend'].astype(int)


In [16]:
df.head()

,app_name,item_id,user_id,recommend,interaction
0,Carmageddon Max Pack,282010,InstigatorAU,True,1
1,Half-Life,70,EizanAratoFujimaki,True,1
2,Half-Life,70,GamerFag,True,1
3,Half-Life,70,76561198020928326,True,1
4,Half-Life,70,Bluegills,True,1


#### Crear la Matriz de Utilidad

In [17]:
matriz_utilidad = df.pivot_table(index='user_id', columns='item_id', values='interaction', fill_value=0)

#### Calcular la Similitud entre Usuarios

In [18]:
similitudes = cosine_similarity(matriz_utilidad)
similitudes_df = pd.DataFrame(similitudes, index=matriz_utilidad.index, columns=matriz_utilidad.index)



#### Identificar Usuarios Similares

In [19]:
# Verificar si el usuario existe en el DataFrame
user_id = 'InstigatorAU'
if user_id not in similitudes_df.index:
    print("El ID de usuario proporcionado no existe en la base de datos.")
else:
    # Ordenar a los usuarios por similitud y seleccionar los top 10 más similares (excluyendo al propio usuario)
    usuarios_similares = similitudes_df[user_id].sort_values(ascending=False)[1:11]

#### Generar Recomendaciones

In [20]:
top_n = 5
juegos_recomendados = set()

for usuario_similar in usuarios_similares.index:
    # Juegos que el usuario similar ha recomendado
    juegos_usuario_similar = set(matriz_utilidad.columns[(matriz_utilidad.loc[usuario_similar] > 0)])
    # Juegos que el usuario objetivo ya conoce
    juegos_usuario = set(matriz_utilidad.columns[(matriz_utilidad.loc[user_id] > 0)])
    # Actualizar las recomendaciones con juegos que el usuario similar ha recomendado pero el usuario objetivo no conoce
    juegos_recomendados.update(juegos_usuario_similar.difference(juegos_usuario))

# Limitar a los top N recomendados
juegos_recomendados = list(juegos_recomendados)[:top_n]

#### Obtener Nombres de los Juegos Recomendados

In [21]:
# Obtener los nombres de los juegos recomendados
nombres_juegos = df[df['item_id'].isin(juegos_recomendados)]['app_name'].drop_duplicates().tolist()

In [22]:
nombres_juegos

['Rocket League®',
 'DARK SOULS™ II',
 'No More Room in Hell',
 'Castle Crashers®',
 'Portal']